# Overview of Database

The `DB()` object in the `dl_utils.db` module encapsulates functionality to organize project data and track data transformations as well as analysis. As the size and complexity of a project grows, use of the `DB()` object will ensure that:

1. Project file paths are defined in a well-organized directory hierarchy
2. Project metadata is serialized in a clear and standard format
3. Code to perform data transformations and analysis are well-documented with clear input(s) and output(s)
4. The entire data transformation pipeline can be applied easily to new cohorts

### Table of Contents

* Initialization
* Basic functionality
* Iteration

# Set Up

The following lines of code prepare the requisite environment to run this notebook.

In [ ]:
import os
import pandas as pd
from dl_utils.db import DB

In addition this tutorial assumes that the `bet` example dataset has been downloaded in the `dl_utils/data` folder. If needed, the following lines of code will download the required data:

In [ ]:
from dl_utils import datasets

# --- Set paths
DAT_PATH = '../../../data'
CSV_PATH = '{}/bet/csvs/db-all.csv.gz'.format(DAT_PATH)
YML_PATH = '{}/bet/ymls/db-all.yml'.format(DAT_PATH)

# --- Download data
if not os.path.exists(CSV_PATH):
    datasets.download(name='bet', path=DAT_PATH)

# Initialization

All `DB()` object data including filenames and raw values are stored in a `*.csv` (or `*.csv.gz`) file. If necessary, all `DB()` object metadata such as filename directory roots, filename patterns or method definitions are stored in a `*.yml` file. Either file type may be passed directly into the `DB(...)` constructor to create a new object.

### Creating from a `*.csv` file

All underlying raw `DB()` data is stored in a `*.csv` file. Each row in the `*.csv` file represents a single exam. Each column in the `*.csv` file may be one of three different types: sid, fnames, header.

An example template `*.csv` file is shown here:

```
sid           fname-dat       fname-lbl       hemorrhage
exam-id-000   /000/dat.hdf5   /000/lbl.hdf5   True
exam-id-001   /001/dat.hdf5   /001/lbl.hdf5   False
exam-id-002   /002/dat.hdf5   /002/lbl.hdf5   True
...           ...             ...             ...
```

#### sid (required)

Exactly one column in the `*.csv` file must be named `sid` and be populated with a **unique** study ID for each exam (row). The `sid` may be either numeric or alphanumeric in content.

#### fnames (optional)

If a project utilizes one or more serialized data volumes, the file paths should be maintained in columns specified with a `fname-` prefix (e.g. `fname-dat` and `fname-lbl` as above). Files may be listed using either complete or relative paths, or using a number of keywords. See `*.yml` configuration below for more information.

#### header (optional)

All other data for a project should be maintained in the remaining columns of the `*.csv` file (e.g. not `sid` and not prefixed with `fname-`). It is best practice to serialize a single value per column (e.g. either a numeric value or string), rather than storing multiple values as an object. 

In [ ]:
# --- Create from *.csv file
db = DB(CSV_PATH)

### Creating from a `*.yml` file

In addition to the raw `DB()` data stored in a `*.csv` file, various metadata that defines `DB()` behavior may also be specified in a corresponding `*.yml` file. 

An example template `*.yml` file is shown here:

```yml
files: 
  csv: /csvs/db-all.csv.gz
  yml: /ymls/db-all.yml
paths: 
  data: /path/to/data
  code: /path/to/code
sform: {}
query: {}
fdefs: []

```

#### files and paths (required)

To facilitate transfer of code and data, relative paths are stored in the `files` variable, with path roots stored in the `pqths` variable. Thus:

```python
paths['code'] + files['csv'] # complete path to *.csv file
paths['code'] + files['yml'] # complete path to *.yml file
```

Additionally, `paths['data']` represents the root directory for serialized data volumes.

#### sform (optional)

There are a two different methods to store files paths in the `*.csv` table (in columns prefixed with `fname-`). As above, the simplest method is to use the complete file path name. Alternatively, the `sform` dictionary may be set with key-values pairs where the key represents a column name and the value represents a Python string format pattern. The Python string format pattern may use one of three different keywords:

* *root*: the data root directory (`paths['data'` as above)
* *curr*: the current contents stored in the `*.csv` file
* *sid*: the current exam study ID

Consider the following examples (using the same template `*.csv` as above):

##### Example 1

```yml
sform:
  dat: '{root}/{curr}'
  lbl: '{root}/{curr}`
```

... would be expanded to ...

```
sid           fname-dat                    fname-lbl       
exam-id-000   /path/to/data/000/dat.hdf5   /path/to/data/000/lbl.hdf5
exam-id-001   /path/to/data/001/dat.hdf5   /path/to/data/001/lbl.hdf5
exam-id-002   /path/to/data/002/dat.hdf5   /path/to/data/002/lbl.hdf5
...           ...             ...             ...
```

##### Example 2

```yml
sform:
  dat: '{root}/{sid}/dat.hdf5'
  lbl: '{root}/{sid}/lbl.hdf5`
```

... would be expanded to ...

```
sid           fname-dat                            fname-lbl       
exam-id-000   /path/to/data/exam-id-000/dat.hdf5   /path/to/data/exam-id-000/lbl.hdf5
exam-id-001   /path/to/data/exam-id-001/dat.hdf5   /path/to/data/exam-id-001/lbl.hdf5
exam-id-002   /path/to/data/exam-id-002/dat.hdf5   /path/to/data/exam-id-002/lbl.hdf5
...           ...             ...             ...
```

#### query (optional)

As an alternative to manually identifying the relevant file paths, a simple query can configured to automatically find (and update) the requested data. The query dictinoary is defined simply using a root data directory and one or more matching suffix patterns. 

Consider the following example:

```yml
query:
  root: /path/to/data
  dat: dat.hdf5
  lbl: lbl.hdf5
```

In this scenario:

* column `fname-dat`: populated with results from `glob.glob('/path/to/data/**/dat.hdf5')`
* column `fname-lbl`: populated with results from `glob.glob('/path/to/data/**/lbl.hdf5')`

Note that corresponding `dat.hdf5` and `lbl.hdf5` files for the same exam are expected to be in the **same subdirectory**.

#### fdefs (optional)

See notes below.

In [ ]:
# --- Create from *.yml file
db = DB(YML_PATH)

# Basic functionality

Upon creating a `DB()` object, the underlying data structure is split into two separate `pandas` DataFrames, `db.fnames` and `db.header` (each DataFrame is an attribute of the main `DB()` object). The `db.fnames` DataFrame comprises of all `*.csv` columns prefixed with `fname-` (with the `fname-` prefix itself removed upon import); the `db.header` DataFrame contains all remaining columns.

In [ ]:
# --- Inspect fnames and header
assert type(db.header) is pd.DataFrame
assert type(db.fnames) is pd.DataFrame

# --- Ensure all five exams are available
assert db.fnames.shape[0] == 5
assert db.header.shape[0] == 5

## db.fnames

The `db.fnames` attribute is implemented via a Pandas DataFrame. The DataFrame index and columns represent the individual exam study IDs (`sid`) and different filenames (`fname-`), respectively.

In [ ]:
# --- db.fnames
db.fnames.index
db.fnames.columns

# --- db.fnames shape (row x columns)
db.fnames.shape

Some useful functions:

In [11]:
# --- Check to see if fnames exist (all columns)
db.exists()

# --- Check to see if fnames exist (specified column)
db.exists(cols=['dat'])

# --- Return fully expanded fnames of a single row (specified by sid)
fnames = db.fnames_expand_single(sid='ID_2e28736ab7')

# --- Return fully expanded fnames of a single row (specified by index)
fnames = db.fnames_expand_single(index=0)

## db.header

The `db.header` attribute is implemented via a Pandas DataFrame. The DataFrame index and columns represent the individual exam study IDs (`sid`) and different exam metadata, respectively.

In [ ]:
# --- db.fnames
db.header.index
db.header.columns

# --- db.fnames shape (row x columns)
db.header.shape

Note that by the index (and number of rows) for `db.fnames` and `db.header` must always be indentical:

In [ ]:
assert (db.fnames.index == db.header.index).all()

To return an entire combined row of data (fully expanded filenames and header metadata), use the `db.row(...)` method:

In [ ]:
# --- Return complete row of data (specified by sid)
row = db.row(sid='ID_2e28736ab7')

# --- Return complete row of data (specified by index)
row = db.row(index=0)

# Iteration

Perhaps the most common task in data science involves iterating through a dataset and applying a number of different functions or transformations to the cohort. Accordingly, the `DB()` object is optimized specifically for this type of workflow pattern.

The most simple method to manually iterate through a dataset is to use the `db.cursor(...)` Python generator. In it's most simple use case, the underlying `db.fnames` and `db.header` DataFrames are accessed efficiently using the `itertuples()` method, with fnames expanded based on string format patterns (`db.sform`) as needed. By default, a progress bar is printed (although this may be turned off with the `verbose=False` flag).

In [ ]:
for sid, fnames, header in db.cursor():
    # --- Perform data transformation and analysis here
    pass

### Iterate through partial dataset

As needed either a binary mask or an array of indices may be provided to iterate through only a portion of the dataset. A common use case is to keep track of some useful marker in `db.header` (e.g. `cohort-positive` for all exams with a positive finding), and to use that header column as a mask for iteration.

In [ ]:
# --- Iterate through every exam with a sid ending in '7'
mask = [i[-1] == '7' for i in db.fnames.index]
for sid, fnames, header in db.cursor(mask=mask):
    pass

# --- Iterate through the 0, 2 and 4 exams
indices = [0, 2, 4]
for sid, fnames, header in db.cursor(indices=indices):
    pass

### Iterate using multiple processes

Given the highly parallel nature of iteration, oftentimes it may be useful to split up the task into separate processes. Given the single-threaded nature of the Python interpreter, the easiest way to implement parallel processing is to run multiple jobs in separate Python processes. 

The `DB()` facilitates this process by allowing the user to separate up the underlying data into `n` evenly-sized splits (specified by the `splits` argument). For any given individual Python process, the current data split can be identified by either by setting a shell environment variable `DL_SPLIT` prior to running the Python process, or by manually passing an additional `split` argument; note the former method is the recommended approach.

#### Example 1: set current split using shell environment variable

In [ ]:
# --- Iterate through split 1 of 2 (in file named data.py)
for sid, fnames, header in db.cursor(splits=2):
    pass

Then, from the command line:

```$ DL_SPLIT=0 python data.py```

#### Example 2: set current split using Python variable

In [ ]:
# --- Iterate through split 1 of 2
for sid, fnames, header in db.cursor(split=0, splits=2):
    pass

## Custom Functions

When iterating through a dataset, the most common initial processing step is to load one or more serialized data files. In addition, upon completion of data transformation and/or analysis, the most common final processing step is to either serialize new data volumes and/or store intermediate results. To provide a uniform template for these operations, and to facilitate an organized, explicitly documented approach to this complex process, the `DB()` object supports a high-level framework for performing these tasks.

The general approach involves the following steps:

* identify column(s) to use as function input(s)
* identify column(s) to use as function output(s)
* define function in separate Python file

Here, any arbitrary function that performs some combination of data transformation, processing and/or analysis across the dataset may be considered. Once these key attributes have been determined, the function definition and parameters are added to the `fdefs` list variable in the database `*.yml` file. 

For illustration purposes, let us consider the following `*.yml` file:

```yml
sform:
  - kwargs:
      arr: dat-org
    lambda: null
    python: 
      file: /defs/xform/data.py
      name: create_samp
    return:
      arr: dat-256
```

Let us also consider the following `/defs/xform/data.py` file:

```python
def create_samp(arr):
    
    return {'arr': arr[::2, ::2, ::2]}
```

Using these examples, let us examine the following step-by-step process for defining the custom function.

1. Identify the input argument(s) of the function in `kwargs`

The input argument(s) to the function are defined in the `kwargs` dict within the `sform` variable. Each key-value pair is coded such that the key represents the keyword-delimited argument name in the custom function signature, and the value represents the source of populated data. Note that values can be derived from the following sources:

* column in `db.fnames`: the value will be replaced with the loaded data
* column in `db.header`: the value will be passed directly into the function
* constant: if the value does not match any column, then it will be passed as a constant

In the above example, if there is a column in `db.fnames` named `dat-org`, then the corresponding file will be loaded and passed as `arr` into the custom function. 

2. Identify the output argument(s) of the function in `return`

The return of all custom functions must be a Python dictionary. For any dictionary item that is to be serialized (as a file) or stored (in a column within `db.header`), an appropriate mapping must be made in the `return` dict within the `sform` variable. Note that as before, the values of the dict can reference one of the following sources:

* column in `db.fnames`: the data will be serialized at the specified file path
* column in `db.header`: the data will be stored in corresponding `db.header` column

In the abovoe example, if there is a column in `db.fnames` named `dat-256`, then the corresponding returned array in `{'arr': ...}` will be serialized at the file path in the `db.fnames['dat-256']` column.

3. Define the function in either `lambda` or `python`

Most commonly, the function code itself will be implemented in a separate Python file. The location of the file is defined by the `paths['code']` directory root as well as the `python['file']` dict in the `sform` variable. The method name of the corresponding function within the Python file is defined in the `python['name']` variable.

### Running custom functions

To run a custom function, simply use the `db.refresh(...)` method to update all row contents. The following arguments should be passed:

* **cols**: a list of column name(s) to update; based on these column names, all matching functions (e.g. those whose return(s) match the specified names) will be executed
* **load**: the method used to load files (e.g. `io.load`); this method may be custom function depending on underlying file format; if no function is passed, all matching file names will simply be passed as strings

See examples below:

```python
from dl_utils import io

# --- Run function that generates dat-256 output
db.refresh(cols=['dat-256'], load=io.load)
```

Note that the `db.refresh(...)` method accepts all of the kwargs described above in `db.cursor(...)` to iterate using multiple processes, masks, etc.